In [1]:
import pandas as pd

## Create the ground truth

In [2]:
test_data = pd.DataFrame.from_csv('../data/subset_test.csv')

In [3]:
test_data.head()

,row_id,security_id,p_tcount,p_value,trade_vwap,trade_volume,initiator,transtype1,time1,bid1,...,bid96,ask96,bid97,ask97,bid98,ask98,bid99,ask99,bid100,ask100
254689,254690,18,18917,10969715628,435.1,66,S,Q,15:46:24.722,435.05,...,434.85,434.95,434.85,435.00,434.90,435.0,434.90,435.0,434.85,435.00
500810,500811,18,12917,9792408270,432.0,7661,S,Q,08:32:12.243,431.65,...,431.45,431.90,431.45,431.95,431.45,432.0,431.45,432.0,431.45,432.05
196078,196079,73,14971,12747929651,3152.0,610,B,Q,13:50:48.067,3153.50,...,3151.00,3152.50,3151.00,3152.50,3151.00,3152.5,3151.00,3152.5,3151.00,3152.00
226436,226437,102,13344,8492047588,935.4,45,B,Q,16:03:29.155,935.30,...,935.20,935.90,935.20,935.50,935.20,935.9,935.20,935.9,935.20,935.90
339574,339575,102,17510,11402659276,947.8,500,B,Q,08:42:34.174,945.60,...,948.00,948.80,947.90,948.80,948.00,948.8,948.10,948.8,948.10,948.80


In [4]:
columnsToRetain = ['row_id']

for i in range(52, 101):
    for column in test_data.columns.values:
        if column.endswith(str(i)):
            columnsToRetain.append(column)

In [5]:
ground_truth = test_data[columnsToRetain]

In [6]:
ground_truth.head()

,row_id,bid52,ask52,bid53,ask53,bid54,ask54,bid55,ask55,bid56,...,bid96,ask96,bid97,ask97,bid98,ask98,bid99,ask99,bid100,ask100
254689,254690,435.0,435.15,435.05,435.15,435.05,435.15,435.05,435.2,435.05,...,434.85,434.95,434.85,435.00,434.90,435.0,434.90,435.0,434.85,435.00
500810,500811,431.9,432.10,431.85,432.10,431.85,432.10,431.80,432.1,431.80,...,431.45,431.90,431.45,431.95,431.45,432.0,431.45,432.0,431.45,432.05
196078,196079,3151.5,3153.50,3151.50,3153.50,3151.50,3153.50,3151.50,3153.5,3151.50,...,3151.00,3152.50,3151.00,3152.50,3151.00,3152.5,3151.00,3152.5,3151.00,3152.00
226436,226437,935.2,935.50,935.20,935.50,935.20,935.50,935.20,935.5,935.20,...,935.20,935.90,935.20,935.50,935.20,935.9,935.20,935.9,935.20,935.90
339574,339575,947.2,948.00,947.30,948.00,947.30,948.10,947.30,948.2,947.30,...,948.00,948.80,947.90,948.80,948.00,948.8,948.10,948.8,948.10,948.80


In [7]:
ground_truth.to_csv('../predictions/ground_truth.csv')

## Evaluation

In [7]:
fileToCompare = 'ground_truth' + '.csv'

In [8]:
prediction = pd.DataFrame.from_csv('../predictions/' + fileToCompare)

In [9]:
import math

In [13]:
for column in ground_truth.columns.values:
    if not column in prediction.columns.values:
        raise Exception(column + " missing in prediction csv file " + fileToCompare)

if ground_truth.shape[0] != prediction.shape[0]:
    raise Exception("Wrong number of rows in prediction csv file " + fileToCompare)

if ground_truth.shape[1] != prediction.shape[1]:
    raise Exception("Wrong number of columns in prediction csv file " + fileToCompare)

MSE = 0.0
total_rows = ground_truth.shape[0]
current = 1
count = 0
for index, row in ground_truth.iterrows():
    row_id = row['row_id']
    prediction_row = prediction[prediction['row_id'] == row_id]
    
    if prediction_row.shape[0] != 1:
        raise Exception("Incorrect prediction for row " + str(row_id) + ", in file " + fileToCompare)
    
    for column in ground_truth.columns.values:
        if column != 'row_id':
            pred = prediction_row[column].values[0]
            truth = row[column]

            MSE += ((pred - truth)**2)
            count += 1
            
    print "%d/%d --> %f" %(current, total_rows, MSE/count)
    current += 1

RMSE = math.sqrt(MSE/count)

435.0

435.0
1/27103 --> 0.000000
Root-Mean-Square-Error:  0.0


In [ ]:
print "Root-Mean-Square-Error: ", RMSE